(05:Testing)=
# Testing
<hr style="height:1px;border:none;color:#666;background-color:#666;" />

Testing is an important part of Python package development but one that is often neglected due to the perceived additional workload. However, the reality is quite the opposite! Introducing formal, automated testing into your workflow can have several benefits:

1. **Fewer bugs:** you’re explicitly constructing and testing your code from the viewpoint of a developer and a user;
2. **Better code structure:** writing tests forces you to structure and compartmentalise your code so that it's easier to test and understand;
3. **Easier development:** formal tests will help others (and your future self) add features to your code without breaking the tried-and-tested base functionality.

**Chapter 3: {ref}`03:How-to-package-a-Python`** briefly introduced unit testing in Python package development. This chapter now describes in more detail how to implement formal and automated testing into a Python workflow. We'll discuss how to write tests, how to test efficiently with things like fixtures and parameterizations, and how to calculate how much of your package's source code your tests actually cover.

## Testing in Python

A unit test is a test written to verify that a chunk of code is working as expected. You probably already conduct informal unit tests of your code in your current workflow. In a typical workflow, we write code and then run it a few times in a Python session to see if it's working as we expect. This is informal testing, sometimes called "manual testing" or "exploratory testing". The whole idea behind automated testing is to define a formal, efficient and reproducible unit testing procedure to test your code.

There are a few testing frameworks available in Python. The two most common are:

1. [`unittest`](https://docs.python.org/3/library/unittest.html)
2. [`pytest`](https://docs.pytest.org/en/latest/)

The `unittest` framework is part of the standard Python library and has been around for a while so it is used by plenty of open source projects. However, we will be using `pytest` as our testing framework in this book. `pytest` is not part of the standard Python library but it is commonly used in the Python packaging community. It is fully-featured, simple and intuitive to use (especially for beginners), and is supported and extendable by a large ecosystem of plugins.

## Testing workflow

The basic testing workflow comprises three key parts:

1. Create the test file and directory structure;
2. Write and run tests; and,
3. Determine test code coverage.

We'll describe each step in a little more detail below, but be aware that the whole testing workflow is often an iterative procedure. As you develop your code, add features, and find bugs, you'll be writing additional tests, checking the code coverage, and writing more tests. In the rest of this chapter we'll be building on the `partypy` package we created in **Chapter 3: {ref}`03:How-to-package-a-Python`**.

## Test structure

`pytest` will run all files of the form *`test_*.py`* or *`*_test.py`* in the current directory and its subdirectories. Standard practice for Python packages is to place test modules in a *`tests`* sub-directory within the root package directory. The `cookiecutter` [template](https://github.com/UBC-MDS/cookiecutter-ubc-mds) that we downloaded and used in **Chapter 3: {ref}`03:How-to-package-a-Python`** to build our `partypy` package automatically created this directory structure for us:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.rst
├── CONDUCT.rst
├── CONTRIBUTING.rst
├── docs
├── LICENSE
├── pyproject.toml
├── README.md
├── src
└── tests
    └── test_partypy.py
```

As discussed back in **Chapter 4: {ref}`04:Package-structure-and-distribution`**, you can also choose to include tests as part of your distribution package (either by placing them in the *`src`* directory or by specifying in the *`pyproject.toml`* that the *`tests`* directory should be included in the distribution package), so that they are available to users when they install the package, but this is less common. 

Large libraries often split tests up into multiple *`test_*.py`* or *`*_test.py`* files within the *`tests`* directory for further organization; for example, it is common to see one test module per package module. However, for simple Python packages a single file will often suffice, which by default is usually named *`test_yourpackagename.py`*.

Recall that in **Chapter 3: {ref}`03:How-to-package-a-Python`** we added the following tests for our `partypy` package:

```python
from partypy.simulate import simulate_party
from partypy.plotting import plot_simulation
import pandas as pd
import altair as alt


def test_simulate_party():
    p_0 = [0, 0, 0]
    p_1 = [1]
    assert isinstance(simulate_party(p_0), pd.DataFrame)
    assert simulate_party(p_0, 10)["Total guests"].sum() == 0
    assert simulate_party(p_1, 10)["Total guests"].sum() == 10


def test_plot_simulation():
    p_0 = [0, 0, 0]
    results = simulate_party(p_0)
    plot = plot_simulation(results)
    assert isinstance(plot, alt.Chart)
    assert plot.mark == "bar"
    assert plot.data["Total guests"].sum() == 0
```

We'll talk about the above syntax in the next section, but for now, we can run those tests by simply typing `pytest` at the command line:

```{note}
We are running this command in the `partypy` virtual environment we created in **Chapter 3: {ref}`03:How-to-package-a-Python`**, which already has `pytest` installed as a development dependency. If you're following along with the example, recall that you can activate this `conda` virtual environment by running `conda activate partypy` in your terminal.
```

```{prompt} bash \$ auto
$ pytest
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
collected 2 items                                                              

tests/test_partypy.py ...                                                [100%]

============================== 2 passed in 0.29s ===============================
```

The output of `pytest` provides some basic system information, along with how many tests were run and what percentage passed. If a test fails, it will output the trace-back of the error, so you can see exactly what line of your test failed (we saw an example of this back in **Chapter 3: {ref}`03:How-to-package-a-Python`**). That's really all there is to the basics of automated testing! In the next section, we'll describe how to write tests using `pytest` in more detail.

## Writing tests

### The basics

As discussed above, we define tests for `pytest` to run as functions that are prefixed with `test_` in files of the form *`test_*.py`* or *`*_test.py`*. The test functions for `partypy` above show examples of this syntax. Within the test function, the test code will be written. Below, we'll describe three common types of tests you might want to use for evaluating your code.

#### Asserting that a statement is true

One of the most common tests you will write will compare the result of your code to a known or expected result with the help of an `assert` statement. We've seen an example of this previously in our `test_simulate_party()` function:

```python
def test_simulate_party():
    p_0 = [0, 0, 0]  # 3 guests with probability 0 of attending
    p_1 = [1]        # 1 guest with probability 1 of attending
    assert isinstance(partypy.simulate_party(p=p_0), pd.DataFrame)
    assert partypy.simulate_party(p=p_0, n_simulations=10)["Total guests"].sum() == 0
    assert partypy.simulate_party(p=p_1, n_simulations=10)["Total guests"].sum() == 10
```

In the example above, we are asserting that:

1. The `simulate_party()` function returns an object with the type `pd.DataFrame`;
2. Running 10 simulations with a guest attendance of 0 probability results in a total of 0, because no guests attended any of the simulated parties; and,
3. Running 10 simulations with a guest attendance of 1 probability results in a total of 10, because that guest was simulated to be present at all of the 10 simulated parties.

As you can see, in a `pytest` test function you may include one or more `assert` statements. If any of the included `assert` functions fails, the whole test will fail.

The `assert` statement can be used with any statement that evaluates to a boolean (`True`/`False`). You may also follow the `assert` statement with a string that will can return relevant information to the user if the `assert` fails. Consider the following simple example:

```{prompt} python >>> auto
>>> x = 1
>>> assert (x < 0), 'x is not negative.'
```

```python
AssertionError: x is not negative.
```

#### Assert that numbers are approximately equal

Due to the limitations of floating-point arithmetic in computers, numbers that we would expect to be equal are sometimes not (read more about that interesting topic in the [Python documentation](https://docs.python.org/3/tutorial/floatingpoint.html)):

```{prompt} python >>> auto
>>> 0.1 + 0.2 == 0.3
```

```python
False
```

As a result, when working with floating-point numbers it is common to write tests that determine if numbers are approximately equal. For this we can use the `approx` function from `pytest`.

```{prompt} python >>> auto
>>> from pytest import approx
>>> 0.1 + 0.2 == approx(0.3)
```

```python
True
```

Evaluating numbers as approximately equal can also be useful if your function has some stochastic element in it. You can use the `abs` and `rel` arguments to specify exactly how much absolute or relative error you want to allow in your test. For example, allowing a relative error of 0.1 (10%), will determine 11 is approximately equal to 10 (because 10 +/− 10% gives us a range of 9 to 11):

```{prompt} python >>> auto
>>> 11 == approx(10, rel=0.1)
```

```python
True
```

If we used the absolute error in the above statement, we'd get a `False` result, because 11 is not in the range of 10 +/− 0.1:

```{prompt} python >>> auto
>>> 11 == approx(10, abs=0.1)
```

```python
False
```

#### Assert that an exception is raised

Consider the `simulate_party` function of our `partypy` package:

```python
import numpy as np
import pandas as pd


def simulate_party(p, n_simulations=500):
    """Simulate guest attendance at a party.

    The attendance of each guest is treated as a Bernoulli random variable
    with probability of attendance `p`. The total number of attending guests
    is summed up for each `n_simulations`.

    Parameters
    ----------
    p : float or array_like of floats
        Probability of guest attendance, >= 0 and <=1.
    n_simulations : int, optional
        Number of simulations to run. By default, 500.

    ...docstring hidden...
    """
    result = np.random.binomial(n=1, p=p, size=(n_simulations, len(p))).sum(axis=1)
    return pd.DataFrame(
        {"Total guests": result, "Simulation": range(1, n_simulations + 1)}
    ).set_index("Simulation")
```

From the docstring we can see that this function expects the user to pass numeric values of `p` between 0 and 1, and values of `n_simulations` greater than 0, but the function does not check that this is the case. Therefore, if we passed in input that violated one of these constraints, like passing a negative value for `n_simulations`, we'd get a generic error message:

```{prompt} python >>> auto
>>> import partypy
>>> partypy.simulate_party(p=[0.1, 0.5, 0.9], n_simulations=-5)
```

```python
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/Users/tomasbeuzen/GitHub/py-pkgs/partypy/src/partypy/simulate.py", line 35, in simulate_party
    result = np.random.binomial(n=1, p=p, size=(n_simulations, len(p))).sum(axis=1)
  File "mtrand.pyx", line 3389, in numpy.random.mtrand.RandomState.binomial
ValueError: negative dimensions are not allowed
```

This error message, bubbling up from the NumPy library that our function depends on, is not very intuitive for our users and we may want to handle this error ourselves and raise a more intuitive error message. Let's modify our `simulate_party()` function to check that `n_simulations` is greater than 0, and if not raise a useful error message:

```python
def simulate_party(p, n_simulations=500):
    """Simulate guest attendance at a party.

    ...docstring hidden...
    """
    if n_simulations < 1:
        raise ValueError("n_simulations must be greater than 0.")
    result = np.random.binomial(n=1, p=p, size=(simulations, len(p))).sum(axis=1)
    return pd.DataFrame(
        {"Total guests": result, "Simulation": range(1, simulations + 1)}
    ).set_index("Simulation")
```

We've used the `ValueError` exception here to indicate that the user has passed a value to `n_simulations` that has the right type but an inappropriate value. There are many other built-in exceptions for particular circumstances and you can read more about exceptions in general in the [Python documentation](https://docs.python.org/3/library/exceptions.html).

We can check that our new error handling is working by starting a new Python session and then re-trying our failed code from before:

```{tip}
Recall that `poetry` installs packages in "editable" mode by default, which means that edits to the source code are immediately reflected in the package without having to reinstall it (i.e., without having to run `poetry install`).
```

```{prompt} python >>> auto
>>> import partypy
>>> partypy.simulate_party(p=[0.1, 0.5, 0.9], n_simulations=-5)
```

```python
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/Users/tomasbeuzen/GitHub/py-pkgs/partypy/src/partypy/simulate.py", line 36, in simulate_party
    raise ValueError("n_simulations to run must be greater than 0.")
ValueError: n_simulations to run must be greater than 0.
```

Great, we've verified that our `simulate_party()` function now throws a more helpful error message for the situation in which a user passes a value < 1 to `n_simulations`. The final thing to do is formalize this as a unit test! The way we can check that our code will raise a particular exception for a given situation is using the `raises` statement of `pytest`. We'll import `raises` at the top of our file *`test_partypy.py`* file and add a new test to our existing `test_simulate_party()` function to demonstrate this:

```python
from partypy.simulate import simulate_party
from partypy.plotting import plot_simulation
import pandas as pd
import altair as alt
from pytest import raises  


def test_simulate_party():
    p_0 = [0, 0, 0]
    p_1 = [1]
    assert isinstance(simulate_party(p_0), pd.DataFrame)
    assert simulate_party(p_0, 10)["Total guests"].sum() == 0
    assert simulate_party(p_1, 10)["Total guests"].sum() == 10
    with raises(ValueError):  # here is our new test
        simulate_party([0], n_simulations=-5)


def test_plot_simulation():
    p_0 = [0, 0, 0]
    results = simulate_party(p_0)
    plot = plot_simulation(results)
    assert isinstance(plot, alt.Chart)
    assert plot.mark == "bar"
    assert plot.data["Total guests"].sum() == 0
```

In the new test above, we purposefully pass a negative value to `n_simulations` which we expect to raise a `ValueError` exception. If the exception is raised, our test will pass. Let's check that everything is working as expected and our tests are passing by running `pytest` at the terminal:

```{prompt} bash \$ auto
$ pytest
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
collected 2 items                                                              

tests/test_partypy.py ...                                                [100%]

============================== 2 passed in 0.29s ===============================
```

We've just made two important changes to our package - we've updated the `simulate_party()` function to check that `n_simulations` is greater than 0, and we've written a unit test to verify this functionality is working. Let's commit these changes to local version control (we'll push these, and more, changes to our remote repository at the end of this chapter):

```{note}
Recall that we are making changes to our `partypy` package on the `dev` branch of our version controlled project. You can check what branch you're on by running: `git branch --show-current`. We'll consolidate the changes we're making on this branch into a new version of our `partypy` package in **Chapter 7: {ref}`07:Releasing-and-versioning`**.
```

```{prompt} bash \$ auto
$ git add src/partypy/simulate.py tests/test_partypy.py
$ git commit -m "add n_simulations value check to simulate_party and a unit test"
```

You can write as many tests as you like for your code, but remember to focus on writing tests that evaluate one individual unit of code at a time. Ideally, your tests will cover all the core functionality and anticipated uses of your code. Later in this chapter we'll show how to determine how much of your code is being "covered" by your tests. But first, let's look at some more advanced ways to write and organize your tests.

### Advanced testing

You can populate your test module(s) with as many individual test functions as you like, however as the number of tests grows it is helpful to streamline and organize your tests in a more efficient and accessible manner. Fixtures, parameterizations, and classes in `pytest` are a few of the key things that can help here.

#### Fixtures

You may have noticed some repetition in our `partypy` tests; we define variables `p_0` and `p_1` multiple times to use in different tests. This is inefficient and violates the "Don't repeat yourself" (DRY) principle of software development and is where fixtures can come in handy. Fixtures are useful for sharing resources (functions or data) with test functions, and can be used by importing the `fixture` decorator from `pytest`. To demonstrate the functionality of a fixture, it is perhaps easiest to see a simple example.

Below we import the `fixture` decorator from `pytest` at the top of the module and then create a fixture called `test_data()` which returns a dictionary of data (`p_0` and `p_1`) that we would like to use with our test functions (your fixture could of course return any object). This fixture can be passed to our test functions as an argument and then used within the function, meaning that we don't have to repeatedly define our data in each test:

```python
from partypy.simulate import simulate_party
from partypy.plotting import plot_simulation
import pandas as pd
import altair as alt
from pytest import raises, fixture


@fixture()
def test_data():
    return {
        "p_0": [0, 0, 0],  # 3 guests with probability 0 of attending
        "p_1": [1],        # 1 guests with probability 1 of attending
    }


def test_simulate_party(test_data):
    assert isinstance(simulate_party(test_data["p_0"]), pd.DataFrame)
    assert simulate_party(test_data["p_0"], 10)["Total guests"].sum() == 0
    assert simulate_party(test_data["p_1"], 10)["Total guests"].sum() == 10
    with raises(ValueError):
        simulate_party([0], n_simulations=-5)


def test_plot_simulation(test_data):
    results = simulate_party(test_data["p_0"])
    plot = plot_simulation(results)
    assert isinstance(plot, alt.Chart)
    assert plot.mark == "bar"
    assert plot.data["Total guests"].sum() == 0
```

Overall, using fixtures can make the testing workflow more efficient and reproducible. You can read more about the functionality and use-cases of fixtures in the [`pytest` docs](https://pytest.readthedocs.io/en/2.8.7/fixture.html#). For now, let's also add the above change to local version control:

```{prompt} bash \$ auto
$ git add tests/test_partypy.py
$ git commit -m "using data fixture in tests"
```

#### Parameterizations

Parameteriziations can be useful for running a test multiple times using different inputs. This functionality is facilitated by the `mark.parametrize` decorator in `pytest` and is also helpfully illustrated by example.

Below is an example of parameterizing a test with two different scenarios (we won't make any changes to our source code here, we will just demonstrate the concept of parameterizations). The first argument passed to `@mark.parametrize` is a string that defines the variables to be used in the test function (`p`, `n`, `expected`), and the second argument is a list of tuples, where each tuple contains the three elements (in order: `p`, `n`, `expected`) required to run the test. In our first test below we check that

1. Running 10 simulations with a guest attendance of 0 probability results in a total of 0, because no guests attended any of the simulated parties; and,
2. Running 10 simulations with a guest attendance of 1 probability results in a total of 10, because that guest was simulated to be present at all of the 10 simulated parties.


```python
from partypy.simulate import simulate_party
from pytest import mark


@mark.parametrize("p,n,expected", [([0], 10, 0), ([1], 10, 10)])
def test_simulate_party(p, n, expected):
    assert simulate_party(p, n)["Total guests"].sum() == expected
```

If we ran this test module with `pytest`, the output would tell us that it ran two tests (it ran one test two times with different inputs):

```{prompt} bash \$ auto
$ pytest
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.2, pytest-6.2.3, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
collected 2 items                                                              

tests/test_partypy.py ...                                                [100%]

============================== 2 passed in 0.34s ===============================
```

For more complex testing workflow, it is also possible to combine fixtures and parameterizations, as can be read about in the `pytest` [documentation](https://docs.pytest.org/en/6.2.x/fixture.html#fixture-parametrize).

#### Classes and test organization

Classes can be used in `pytest` as a way of grouping related tests together. For example, you may wish to group all the test for a specific module of your package, or you may have multiple tests for a particularly complex function which you'd like to group together. You can define a test class in your test module by creating a `class` with a name prefixed with "Test". You can then nest all relevant test functions within this class as is shown below:

```python
class Test_simulate:
    def test_simulate_party(self):
        ...
    
class Test_plotting:
    def test_plot_simulation(self):
        ...
    
class Test_datasets:
    def test_load_party(self):
        ...
```

On the other hand, it's common for developers to simply create one test module per package module for organizational purposes, e.g.:

```
partypy
├── ...
└── tests
    ├── test_simulate.py
    ├── test_plotting.py
    └── test_datasets.py
```

The organizational structure you choose to use is mostly personal preference. However, it's worth mentioning that classes in `pytest` are not only useful organizational tools, but they can also be used for more advanced workflows, such as to share parametrizations among all tests in the class, as can be read about in the `pytest` [documentation](https://docs.pytest.org/en/6.2.x/contents.html).

#### Further reading

For beginner to intermediate Python packagers the methods discussed above will more than suffice for your package testing needs. However we've only scratched the surface of the `pytest` iceberg and it's possible to combine classes, fixtures, parameterizations, and much more to help build efficient, streamlined test workflows which can be useful for more complex projects. As your testing workflow expands, we highly recommend taking a look at the `pytest` [documentation](https://docs.pytest.org/en/latest/) to learn more about its capabilities and best practices.

### When to write your tests

Whether you should write your tests before you code, after you code, or somewhere in between is a topic of much debate and overall this choice may come down to personal preference, experience, resource availability, and code complexity. While there's no "correct" testing workflow for everyone, we encourage you to have a go at Test-Driven-Development (TDD) - that is, writing your tests before you code. While this may seem a little counter-intuitive at first, the TDD workflow can have many benefits:

- You will better understand exactly what code you need to write;
- You are forced to write tests upfront;
- You won't encounter large time-consuming bugs down the line; and,
- It helps to keep your workflow manageable by focusing on small, incremental code improvements and additions.

Overall, TDD requires a larger upfront cost (of time and effort), but can help keep your workflow manageable and error-free and save you lots of time down the road. In our experience, errors found earlier are much easier to fix than errors found later, i.e., the age-old proverb, *prevention is better than cure*.

## Code coverage

The tests you write should "cover" the majority of your code, that is, your tests should run most or all of your code at least once - there are certainly exceptions to this, but the general idea is to have your tests cover the core functionality of your package. We refer to this as "code coverage" and there is a useful extension to `pytest` called `pytest-cov` which we can use to automatically determine how much coverage our tests have.

Continuing with the `partypy` package that we have developed throughout this book, let's use `poetry` to add `pytest-cov` as a development dependency of our package:

```{prompt} bash \$ auto
$ poetry add --dev pytest-cov
```

We can then determine the coverage of our tests by running the following command which tells `pytest-cov` to determine the coverage our tests have of the `partypy` package:

```{prompt} bash \$ auto
$ pytest --cov=partypy
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
plugins: cov-2.12.1
collected 2 items                                                                                                                      

tests/test_partypy.py ..                                                                                                         [100%]

---------- coverage: platform darwin, python 3.9.6-final-0 -----------
Name                      Stmts   Miss  Cover
---------------------------------------------
src/partypy/__init__.py       5      0   100%
src/partypy/datasets.py       6      3    50%
src/partypy/plotting.py       4      0   100%
src/partypy/simulate.py       7      0   100%
---------------------------------------------
TOTAL                        22      3    86%

============================== 2 passed in 0.46s ===============================
```

The output summarizes the coverage of the individual modules in our `partypy` package. This is a helpful high-level summary of our test coverage. For example, we can see that our current tests only cover 50% of our *`datasets.py`* module which we created back in **Chapter 4: {ref}`04:Package-structure-and-distribution`**!

If we want to see a more detailed output of exactly what lines of code our tests are missing, `pytest-cov` can generate a useful HTML report using the `--cov-report html` argument:

```{prompt} bash \$ auto
$ pytest --cov=partypy --cov-report html
```

The report will be available at *`htmlcov/index.html`* and will look as below:

```{figure} images/test-report-1.png
---
width: 100%
name: 05-test-report-1
alt: HTML test report.
---
HTML test report.
```

We can click on elements of the report, like the *`datasets.py`* module, to see exactly what lines are tests are hitting/missing:

```{figure} images/test-report-2.png
---
width: 100%
name: 05-test-report-2
alt: Detailed view of the datasets module in the HTML report.
---
Detailed view of the datasets module in the HTML report.
```

Let's add a test for the `load_party()` function of our *`datasets.py`* module to our test file *`test_partypy.py`*, the full file of which now looks like this:

```python
from partypy.simulate import simulate_party
from partypy.plotting import plot_simulation
from partypy.datasets import load_party
import pandas as pd
import altair as alt
from pytest import raises, fixture


@fixture()
def test_data():
    return {
        "p_0": [0, 0, 0],  # 3 guests with probability 0 of attending,
        "p_1": [1],  # 1 guests with probability 1 of attending,
    }


def test_simulate_party(test_data):
    assert isinstance(simulate_party(test_data["p_0"]), pd.DataFrame)
    assert simulate_party(test_data["p_0"], 10)["Total guests"].sum() == 0
    assert simulate_party(test_data["p_1"], 10)["Total guests"].sum() == 10
    with raises(ValueError):
        simulate_party([0], n_simulations=-5)


def test_plot_simulation(test_data):
    results = simulate_party(test_data["p_0"])
    plot = plot_simulation(results)
    assert isinstance(plot, alt.Chart)
    assert plot.mark == "bar"
    assert plot.data["Total guests"].sum() == 0


def test_load_party():
    df = load_party()
    assert isinstance(df, pd.DataFrame)  # function should return a pandas dataframe
    assert len(df) == 100  # data should contain 100 example guests

```

Re-running our tests and calculating our code coverage should hopefully reveal a 100% coverage:

```{prompt} bash \$ auto
$ pytest --cov=partypy
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.2, pytest-6.2.3, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
collected 3 items                                                              

tests/test_partypy.py ...                                                [100%]

---------- coverage: platform darwin, python 3.9.6-final-0 -----------
Name                      Stmts   Miss  Cover
---------------------------------------------
src/partypy/__init__.py       5      0   100%
src/partypy/datasets.py       6      0   100%
src/partypy/plotting.py       4      0   100%
src/partypy/simulate.py       7      0   100%
---------------------------------------------
TOTAL                        22      0   100%

============================== 3 passed in 0.39s ===============================
```

We've now updated our *`test_partypy.py`* file to include tests for the *`datasets.py`* module and we've added `pytest-cov` as a development dependency (which is recorded in *`pyproject.toml`* and *`poetry.lock`*). Let's commit these changes to local version control, and then push all the changes we've made in this chapter to our remote repository:

```{prompt} bash \$ auto
$ git add poetry.lock pyproject.toml
$ git commit -m "add pytest-cov as dev dependency"
$ git add tests/test_partypy.py
$ git commit -m "add tests to cover 'datasets' module"
$ git push
```

### Types of code coverage

The coverage we were calculating above is known as line coverage. It's simply the percentage of all the lines in your code that your tests hit. However, there are various other forms of coverage too. A notable alternative is branch coverage. Branch coverage evaluates how many branches in your code are executed by tests, where a branch is a possible execution path the code can take. For example, an `if` statement allows your code to execute along one of two paths depending on whether the specified conditions is met or not.

Consider the following simple example:

```python
def positive(x):
    if x < 0:
        y = abs(x)
    return y
```

Here's a test for that function:

```python
def test_positive():
    assert positive(-5) == 5
```

We should have 100% line coverage here. Below is the HTML report generated by `pytest` for this scenario:

```{figure} images/test-report-3.png
---
width: 100%
name: 05-test-report-3
alt: HTML report showing 100% line coverage for the positive function.
---
HTML report showing 100% line coverage for the positive function.
```

However, let's take a look at branch coverage. We can determine branch coverage with `pytest` and `pytest-cov` using the `--cov-branch` argument:

```{prompt} bash \$ auto
$ pytest --cov=positive --cov-branch --cov-report html
```

Below is the generated HTML report:

```{figure} images/test-report-4.png
---
width: 100%
name: 05-test-report-4
alt: HTML report showing 83% branch coverage for the positive function.
---
HTML report showing 83% branch coverage for the positive function.
```

The report above shows that all lines of code have been executed, but the branch in our code, defined by the `if` statement has only been partially executed. This shows us that our `positive()` function and `test_positive()` test are missing an important execution path; what happens if `x >= 0`? Our test has only evaluated one of two possible branches defined by the `if` statement, hence the "partial" highlight in the report above. You might expect that the "branch coverage" shown above should be 50%, not 83%, because our test only executed one of two possible paths through the code and that would be true! However, `pytest-cov` takes a slightly different approach to calculating branch coverage; it calculates it as a combination of line executions and branch executions, namely, it is the sum of executed lines and executed branches divided by the the total number of lines and branches. In the example above that would be (4 + 1) / (4 + 2) = 83%. As a result, `pytest-cov` combines line and branch coverage into a single useful metric.

Regardless of the exact method of calculating coverage, the point is that there are different ways to evaluate the "coverage" of your tests which can provide useful insights about how your code is written and how it might fail. A key takeaway is that a 100% score in one metric like line coverage, doesn't mean your package code, of tests are perfect! Line and branch coverage are the two most popular methods of code coverage and will be acceptable for the large majority of Python packagers, however other methods exist that might prove useful in different situations, such as condition coverage, function coverage, mutation coverage, etc.